# Preprocessing and data cleaning

In [100]:
import numpy as np 
import pandas as pd

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pandas.api.types import is_numeric_dtype , is_string_dtype , is_categorical_dtype

import feather

### Load the training data

In [101]:
# load data
url1 = "https://raw.githubusercontent.com/PicoRadia/sdataProjet/main/dataset/Spotify_train_dataset.csv"
df = pd.read_csv(url1)#31728 rows x 20 columns

## Preprocessing :
* One hot encoding for the discrete variables 
* Robust scaler for the continuous variables
* Feature engineering


In [102]:
def preprocess(dff):
    cols = ['type', 'uri', 'track_href', 'analysis_url', 'song_name','type']
    num = [col for col in dff.columns if is_numeric_dtype(dff[col])]
    dscr = [col for col in dff[num].columns if len(dff[num].groupby(col)[col].unique()) < 20]
    conti = [val for val in num if val not in dscr]
    
    contdf = dff[conti].copy()
    cols = contdf.columns
    # convert the array back to a dataframe
    transform = RobustScaler()
    data = transform.fit_transform(contdf)
    
    # convert the array back to a dataframe
    
    contdf = pd.DataFrame(data)
    contdf.columns = cols

    dscrdf = dff[dscr].copy()
    key_ = pd.get_dummies(dscrdf.key, prefix='Key')

    time_signature_ = pd.get_dummies(dscrdf.time_signature, prefix='time_signature_')

    tmp = pd.get_dummies(dff,columns = ['mode'])
    result = pd.concat([dff,key_, time_signature_,tmp[['mode_0','mode_1']]], axis=1,join="inner")
    result["mins"] = dff["duration_ms"] * 0.001 /60
    # result['mins'] = pd.to_datetime(result['mins']).dt.minute
    dff['song_name'] = dff['song_name'].replace(np.nan, 0)
    l3 = []
    for val in dff.song_name:
        if val == 0:
            l3.append(0)
        else:
            l3.append(1)
    result['song_name_nan'] = l3
    l = []
    for row in df['genre'] :
        if row == 'trap' or row =="Dark Trap" or row=="Trap Metal":
            l.append(1)
        else : 
            l.append(0)
    result['has_trap'] = l
    # Has trance
    l1 = []
    for row in df['genre'] :
        if row == 'trance' or row =="psytrance" :
            l1.append(1)
        else : 
            l1.append(0)
    result['has_trance'] = l1
    
    # Has rap
    l2 = []
    for row in df['genre'] :
        if row == 'Rap' or row =="Underground Rap" :
            l2.append(1)
        else : 
            l2.append(0)
    result['has_rap'] = l2

    # dropping duplicates
    result.drop_duplicates(subset ="id", keep = False, inplace = True) 
    # dropping columns
    cols = ['type', 'uri','genre' ,'track_href', 'analysis_url','id' ,'song_name','type','key','time_signature','mode','duration_ms']
    result.drop(cols, axis = 1,inplace=True)
    return result

In [103]:
dff = df.copy()

In [104]:
x = preprocess(dff)

In [105]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24738 entries, 1 to 31727
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   danceability       24738 non-null  float64
 1   energy             24738 non-null  float64
 2   loudness           24738 non-null  float64
 3   speechiness        24738 non-null  float64
 4   acousticness       24738 non-null  float64
 5   instrumentalness   24738 non-null  float64
 6   liveness           24738 non-null  float64
 7   valence            24738 non-null  float64
 8   tempo              24738 non-null  float64
 9   Key_0              24738 non-null  uint8  
 10  Key_1              24738 non-null  uint8  
 11  Key_2              24738 non-null  uint8  
 12  Key_3              24738 non-null  uint8  
 13  Key_4              24738 non-null  uint8  
 14  Key_5              24738 non-null  uint8  
 15  Key_6              24738 non-null  uint8  
 16  Key_7              247

In [109]:
x.tail(10).T

,31717,31718,31719,31720,31721,31722,31723,31725,31726,31727
danceability,0.619000,0.463000,0.66500,0.80400,0.74600,0.384000,0.855000,0.464000,0.683000,0.868000
energy,0.591000,0.830000,0.87800,0.76200,0.87900,0.667000,0.586000,0.838000,0.590000,0.654000
loudness,-8.232000,-8.483000,-4.74500,-13.68900,-4.00600,-8.126000,-7.134000,-6.762000,-4.949000,-4.832000
speechiness,0.062500,0.045000,0.20600,0.07470,0.10100,0.516000,0.167000,0.040100,0.359000,0.076600
acousticness,0.262000,0.101000,0.00325,0.09000,0.00118,0.062100,0.200000,0.000151,0.215000,0.208000
instrumentalness,0.000145,0.000007,0.00000,0.86700,0.00848,0.000000,0.000000,0.756000,0.000000,0.000000
liveness,0.107000,0.286000,0.06630,0.09810,0.18000,0.330000,0.253000,0.067100,0.135000,0.187000
valence,0.214000,0.116000,0.39400,0.66100,0.17100,0.720000,0.751000,0.151000,0.040100,0.299000
tempo,130.624000,149.745000,186.09400,125.00200,174.06900,187.767000,138.132000,173.995000,209.986000,138.020000
Key_0,0.000000,0.000000,0.00000,0.00000,0.00000,1.000000,0.000000,0.000000,1.000000,0.000000


## Saving result in feather format

In [107]:
x.reset_index().to_feather('./data2')

# Conclusion

We now have clean data that we can use to trin our models and that is fast to load too because it's saved in feather format.
We can do some feature engineering to make our mode better.